In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

# Question 3

The purpose of the function tridiagonal_creator is to create a tridiagonal matrix of the form $A = \frac{1}{h^2} \cdot$ $\begin{pmatrix}
2+c_1h^2 & -1 & 0 & ... & 0\\
-1 & 2+c_2h^2 & -1 & ... & 0\\
.\\
.\\
.\\
0 & ... & 0 & -1 & 2+c_Nh^2\\
\end{pmatrix}$
$\newline$
where $c_j = \pi^2$ for all $j$. The function takes in the input $N$, the size of the matrix.

In [8]:
def tridiagonal_creator(N):
    h = 1/N
    A = np.zeros((N,N))
    for j in range(N):
        if (j == 0):
            A[0][0] = 2+(h**2)*(math.pi**2)
            A[0][1] = -1
        elif (j == N-1):
            A[N-1][N-2] = -1
            A[N-1][N-1] = 2+(h**2)*(math.pi**2)
        else:
            A[j][j-1] = -1
            A[j][j] = 2+((h**2)*(math.pi**2))
            A[j][j+1] = -1
    A = np.array(A)*(1/h**2)
    return A

The following function x_creator creates a list of equally spaced nodes $x_j$, where $j = 1,2,..., N$, given input parameter $N$ (number of nodes plus one).

In [154]:
def x_creator(N):
    x_list = []
    h = 1/N
    for j in range(1, N):
        x_list.append(j*h)
    return x_list

The following function f_creator evaluates a given function $f$ at each node given by the input x_list, generated by x_creator function. Here $f(x_j) = 2\pi^2\sin(\pi x_j)$.

In [127]:
def f_creator(x_list):
    f_list = []
    for j in range(len(x_list)):
        f_list.append((2*math.pi**2)*math.sin(math.pi*x_list[j]))
    return f_list

The following function conjugate_gradient implements the conjugate gradient iteration algorithm taught in class, beginning with the starting vector $x^{(0)} = (0, 0, ..., 0)$ such that $x^{(j+1)} = x^{(j)} + t_j \cdot v^{(j)}$, and with the direction vector $v^{(j)}$ being given by $v^{(j)} = r^{(j)} + s_j \cdot v^{(j-1)}$, where $s_j = \frac{<r^{(j)}, Av^{(j-1)}>}{<v^(j-1), Av^{(j-1)}>}$, where $r^{(j)}$ is the residual of the iteration $j$. It takes as input a matrix $A$, a solution vector $b$, and $N$, the number of iterations to be performed.

In [128]:
def conjugate_gradient(A, b, N):
    x = [0]*len(A)
    r = b - np.matmul(A, x)
    v = r
    t = (np.dot(r, r))/(np.dot(r, np.matmul(A, r)))
    for i in range(N):
        x = x + t*v
        r = b - np.matmul(A, x)
        s = (np.dot(r, np.matmul(A, v)))/(np.dot(v, np.matmul(A, v)))
        t = (np.dot(r, r))/(np.dot(r, np.matmul(A, r)))
        v = r + s*v
    return x

The following function l2_error computes the square root of the sum of squared differences between our actual solution and our approximation. Its input parameters are the vector of approximate solutions found by solving the matrix equation $Ax=b$ using our conjugate gradient algorithm, and the vector of actual solutions created by evaluating our actual solution $f(x) = \sin(\pi x)$ at every node in our list of nodes.

In [129]:
def l2_error(real, approx):
    error_list = []
    for i, val in enumerate(real):
        error_list.append((val - approx[i])**2)
    return math.sqrt(sum(error_list))

In [130]:
A = tridiagonal_creator(49)

In [131]:
x_list = x_creator(50)

In [133]:
b = f_creator(x_list)

In [134]:
# actual solution
x = np.linalg.solve(A, b)

In [135]:
# N = 50
x_approx = conjugate_gradient(A, b, 50)

In [136]:
error_50_iter = l2_error(x, x_approx)
print(error_50_iter)

4.90145312323609e-15


In [137]:
# N = 100
x_approx2 = conjugate_gradient(A, b, 100)

In [138]:
error_100_iter = l2_error(x, x_approx2)
print(error_100_iter)

4.880030895530426e-15


In [139]:
# N = 200
x_approx3 = conjugate_gradient(A, b, 200)

In [140]:
error_200_iter = l2_error(x, x_approx3)
print(error_200_iter)

4.90145312323609e-15


It appears that the conjugate gradient method converges to the actual solution $x$ after about 50 iterations, which is what we would expect since our matrix $A$ has dimension 50, and we know conjugate gradient method converges to the exact solution in at most $n$ iterations.

The following function jacobis_method implements the Jacobi iteration for solving a system of linear equations $Ax=b$, where we follow the iteration formula $x^{(k+1)} = M^{-1}Nx^{(k)} + M^{-1}b$, where $M$ is the diagonal of our matrix $A$. It takes as input the matrix $A$, the solution vector $b$, and num_iter, the number of iterations to be performed.

In [141]:
def jacobis_method(A, b, num_iter):
    M = np.identity(len(A))
    di = np.diag_indices(len(A))
    diags = np.diag(A)
    M[di] = diags
    N = M - A
    T = np.matmul(np.linalg.inv(M), N)
    b2 = np.matmul(np.linalg.inv(M), b)
    x = [0]*len(A)
    for i in range(num_iter):
        x = np.matmul(T, x) + b2
    return x

In [148]:
# N = 50
x_approx4 = jacobis_method(A, b, 50)

In [149]:
error_50_iter2 = l2_error(x, x_approx4)
print(error_50_iter2)

4.171086579287603


In [150]:
# N = 100
x_approx5 = jacobis_method(A, b, 100)

In [151]:
error_100_iter2 = l2_error(x, x_approx5)
print(error_100_iter2)

3.4101470840429675


In [152]:
# N = 200
x_approx6 = jacobis_method(A, b, 200)

In [153]:
error_200_iter2 = l2_error(x, x_approx6)
print(error_200_iter2)

2.279401994991287


It appears that the Jacobi iteration converges far slower than the conjugate gradient method.